In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from covid19model.models.models import COVID19_SEIQRD_spatial_stratified_rescaling
from covid19model.models.utils import initialize_COVID19_SEIQRD_stratified_vacc

# Import time-dependent parameter functions for resp. P, Nc, alpha, N_vacc, season_factor
from covid19model.models.time_dependant_parameter_fncs import make_mobility_update_function, \
                                                          make_contact_matrix_function, \
                                                          make_VOC_function, \
                                                          make_vaccination_function, \
                                                          make_seasonality_function

# Import packages containing functions to load in data used in the model and the time-dependent parameter functions
from covid19model.data import mobility, sciensano, model_parameters, VOC

In [ ]:
# Population size, interaction matrices and the model parameters
# initN, Nc_dict, params, CORE_samples_dict = model_parameters.get_COVID19_SEIQRD_parameters(spatial='prov')
initN, Nc_dict, params, CORE_samples_dict = model_parameters.get_COVID19_SEIQRD_parameters()

# Variants of concern
VOCs = ['WT', 'abc', 'delta']
VOC_logistic_growth_parameters, VOC_params = model_parameters.get_COVID19_SEIQRD_VOC_parameters(initN, params['h'], VOCs=VOCs)
params.update(VOC_params)

# Load and format local vaccination data, which is also under the sciensano object
# public_spatial_vaccination_data = sciensano.get_public_spatial_vaccination_data(update=False)#,agg='prov')
# df_inc = make_vaccination_function(public_spatial_vaccination_data['INCIDENCE']).df

# df_hosp, df_mort, df_cases, df_vacc = sciensano.get_sciensano_COVID19_data()
# df_inc = make_vaccination_function(df_vacc).df

df_inc

In [ ]:
# [none, first, full, waned, booster]
e_s = 1-VOC_params['e_s']
e_i = 1-VOC_params['e_i']
e_h = 1-VOC_params['e_h']


### Wild type values

# hard-coded
onset_days_WT = dict({'E_susc' : {'first' : 21, 'full' : 21, 'booster' : 21},
               'E_inf' : {'first' : 14, 'full' : 14, 'booster' : 14},
               'E_hosp' : {'first' : 21, 'full' : 21, 'booster' : 21}})

# E_init is the value of the previous waned vaccine
E_init_WT = dict({'E_susc' : {'first' : e_s[0,0], 'full' : e_s[0,1], 'booster' : e_s[0,3]},
               'E_inf' : {'first' : e_i[0,0], 'full' : e_i[0,1], 'booster' : e_s[0,3]},
               'E_hosp' : {'first' : e_h[0,0], 'full' : e_h[0,1], 'booster' : e_h[0,3]}})
E_best_WT = dict({'E_susc' : {'first' : e_s[0,1], 'full' : e_s[0,2], 'booster' : e_s[0,4]},
               'E_inf' : {'first' : e_i[0,1], 'full' : e_i[0,2], 'booster' : e_i[0,4]},
               'E_hosp' : {'first' : e_h[0,1], 'full' : e_h[0,2], 'booster' : e_h[0,4]}})
E_waned_WT = dict({'E_susc' : {'first' : e_s[0,1], 'full' : e_s[0,3], 'booster' : e_s[0,4]},
               'E_inf' : {'first' : e_i[0,1], 'full' : e_i[0,3], 'booster' : e_i[0,4]},
               'E_hosp' : {'first' : e_h[0,1], 'full' : e_h[0,3], 'booster' : e_h[0,4]}})

### alpha/beta/gamma values

# hard-coded
onset_days_abc = dict({'E_susc' : {'first' : 21, 'full' : 21, 'booster' : 21},
               'E_inf' : {'first' : 14, 'full' : 14, 'booster' : 14},
               'E_hosp' : {'first' : 21, 'full' : 21, 'booster' : 21}})

# E_init is the value of the previous waned vaccine
E_init_abc = dict({'E_susc' : {'first' : e_s[1,0], 'full' : e_s[1,1], 'booster' : e_s[1,3]},
               'E_inf' : {'first' : e_i[1,0], 'full' : e_i[1,1], 'booster' : e_s[1,3]},
               'E_hosp' : {'first' : e_h[1,0], 'full' : e_h[1,1], 'booster' : e_h[1,3]}})
E_best_abc = dict({'E_susc' : {'first' : e_s[1,1], 'full' : e_s[1,2], 'booster' : e_s[1,4]},
               'E_inf' : {'first' : e_i[1,1], 'full' : e_i[1,2], 'booster' : e_i[1,4]},
               'E_hosp' : {'first' : e_h[1,1], 'full' : e_h[1,2], 'booster' : e_h[1,4]}})
E_waned_abc = dict({'E_susc' : {'first' : e_s[1,1], 'full' : e_s[1,3], 'booster' : e_s[1,4]},
               'E_inf' : {'first' : e_i[1,1], 'full' : e_i[1,3], 'booster' : e_i[1,4]},
               'E_hosp' : {'first' : e_h[1,1], 'full' : e_h[1,3], 'booster' : e_h[1,4]}})

### delta values

# hard-coded
onset_days_delta = dict({'E_susc' : {'first' : 21, 'full' : 21, 'booster' : 21},
               'E_inf' : {'first' : 14, 'full' : 14, 'booster' : 14},
               'E_hosp' : {'first' : 21, 'full' : 21, 'booster' : 21}})

# E_init is the value of the previous waned vaccine
E_init_delta = dict({'E_susc' : {'first' : e_s[2,0], 'full' : e_s[2,1], 'booster' : e_s[2,3]},
               'E_inf' : {'first' : e_i[2,0], 'full' : e_i[2,1], 'booster' : e_s[2,3]},
               'E_hosp' : {'first' : e_h[2,0], 'full' : e_h[2,1], 'booster' : e_h[2,3]}})
E_best_delta = dict({'E_susc' : {'first' : e_s[2,1], 'full' : e_s[2,2], 'booster' : e_s[2,4]},
               'E_inf' : {'first' : e_i[2,1], 'full' : e_i[2,2], 'booster' : e_i[2,4]},
               'E_hosp' : {'first' : e_h[2,1], 'full' : e_h[2,2], 'booster' : e_h[2,4]}})
E_waned_delta = dict({'E_susc' : {'first' : e_s[2,1], 'full' : e_s[2,3], 'booster' : e_s[2,4]},
               'E_inf' : {'first' : e_i[2,1], 'full' : e_i[2,3], 'booster' : e_i[2,4]},
               'E_hosp' : {'first' : e_h[2,1], 'full' : e_h[2,3], 'booster' : e_h[2,4]}})

onset_days = dict({'WT' : onset_days_WT, 'abc' : onset_days_abc, 'delta' : onset_days_delta})
E_init = dict({'WT' : E_init_WT, 'abc' : E_init_abc, 'delta' : E_init_delta})
E_best = dict({'WT' : E_best_WT, 'abc' : E_best_abc, 'delta' : E_best_delta})
E_waned = dict({'WT' : E_waned_WT, 'abc' : E_waned_abc, 'delta' : E_waned_delta})

In [ ]:
##################################################
## Construct time-dependent parameter functions ##
##################################################

# Time-dependent VOC function, updating alpha
VOC_function = make_VOC_function(VOC_logistic_growth_parameters)

VOC_function(pd.Timestamp(2021, 1, 1), 0, 0)[0]

In [ ]:
def waning_exp_delay(days, onset_days, E_init, E_best, E_waned):
    """
    Function that implements time-dependence of vaccine effect.

    Input
    -----
    days : float
        number of days after the novel vaccination
    onset_days : float
        number of days it takes for the vaccine to take full effect
    E_init : float
        vaccine-related rescaling value right before vaccination
    E_best : float
        rescaling value related to the best possible protection by the currently injected vaccine
    E_waned : float
        rescaling value related to the vaccine protection after a waning period.

    Output
    ------
    E_eff : float
        effective rescaling value associated with the newly administered vaccine

    """
    waning_days = 183 # hard-coded to half a year
    if days <= 0:
        return E_init
    elif days < onset_days:
        E_eff = (E_best - E_init)/onset_days*days + E_init
        return E_eff
    else:
        if E_best == E_waned:
            return E_best
        halftime_days = waning_days - onset_days
        A = 1-E_best
        beta = -np.log((1-E_waned)/A)/halftime_days
        E_eff = -A*np.exp(-beta*(days-onset_days))+1
    return E_eff

In [ ]:
def make_rescaling_dataframe(vacc_data, initN, VOC_func):
    """Desired output: DataFrame per week with fractions of vaccination stage per age class and per province NIS
    
    TO DO: resample this dataframe to daily data.
    
    Input
    -----
    
    vacc_data: pd.DataFrame
        Output of make_vaccination_function(vacc_data['INCIDENCE']).df with the default age intervals
    initN: pd.DataFrame
        Output of model_parameters.get_COVID19_SEIQRD_parameters(spatial='prov')
        
    Output
    ------
    
    df_new : pd.DataFrame
        MultiIndex DataFrame with indices date (weekly), NIS, age, dose.
        Values are 'fraction', which always sum to unity (a subject is in only one of four vaccination stages)
    
    """
    spatial=False
    if 'NIS' in vacc_data.reset_index().columns:
        spatial=True
    
    # set initN column names to pd.Interval objects
    intervals = pd.IntervalIndex.from_tuples([(0,12),(12,18),(18,25),(25,35),(35,45),(45,55),(55,65),(65,75),(75,85),(85,120)], closed='left')
    intervals_str = np.array(['[0, 12)', '[12, 18)', '[18, 25)', '[25, 35)', '[35, 45)', '[45, 55)', '[55, 65)', '[65, 75)', '[75, 85)', '[85, 120)'])
    intervals_dict = dict({intervals_str[i] : intervals[i] for i in range(len(intervals))})
    if spatial:
        initN = initN.rename(columns=intervals_dict)
        initN = initN.unstack().reset_index().rename(columns={0 : 'population'})
    else:
        initN = initN.rename(index=intervals_dict)
        initN = pd.DataFrame(initN).rename(columns={0 : 'population'})
    
    # Name nameless column to 'INCIDENCE' and add 'CUMULATIVE' column
    df_new = pd.DataFrame(vacc_data).rename(columns={0 : 'INCIDENCE'})
    if spatial:
        df_new['CUMULATIVE'] = df_new.groupby(level=[1,2,3]).cumsum()
    else:
        df_new['CUMULATIVE'] = df_new.groupby(level=[1, 2]).cumsum()
    
    # Make cumulative fractions by comparing with relevant initN
    df_new = pd.DataFrame(df_new).reset_index()
    if spatial:
        df_new = df_new.merge(initN, left_on=['NIS', 'age'], right_on=['NIS', 'age_class'])
    else:
        df_new = df_new.merge(initN, left_on=['age'], right_on=['age_class'])
    df_new['fraction'] = df_new['CUMULATIVE'] / df_new['population']
    
    # Start redefining vaccination stage fractions
    df_new = df_new.set_index('date') # make sure we don't get NaN values because of mismatching indices
    df_new_copy = df_new.copy()

    # first-only: dose A (first) - dose B (second)
    df_new.loc[df_new['dose']=='B','fraction'] = (df_new_copy.loc[df_new_copy['dose']=='A','fraction'] \
        - df_new_copy.loc[df_new_copy['dose']=='B','fraction']).clip(lower=0, upper=1)

    # full: dose B (second) + dose C (Jansen) - dose E (booster)
    df_new.loc[df_new['dose']=='C','fraction'] = (df_new_copy.loc[df_new_copy['dose']=='B','fraction'] \
        + df_new_copy.loc[df_new_copy['dose']=='C','fraction'] - df_new_copy.loc[df_new_copy['dose']=='E','fraction']).clip(lower=0, upper=1)
    
    # booster: clip between 0 and 1. This is currently the latest stage
    df_new.loc[df_new['dose']=='E','fraction'] = df_new_copy.loc[df_new_copy['dose']=='E', 'fraction'].clip(lower=0, upper=1)
    
    # none. Rest category. Make sure all exclusive categories adds up to 1.
    df_new.loc[df_new['dose']=='A','fraction'] = 1 - df_new.loc[df_new['dose']=='B','fraction'] \
        - df_new.loc[df_new['dose']=='C','fraction'] - df_new.loc[df_new['dose']=='E','fraction']
    
    ### Make sure all incidence and cumulative data are in the right columns
    # full = second + janssen
    df_new.loc[df_new['dose']=='C', 'INCIDENCE'] += df_new.loc[df_new['dose']=='B', 'INCIDENCE']
    df_new.loc[df_new['dose']=='C', 'CUMULATIVE'] += df_new.loc[df_new['dose']=='B', 'CUMULATIVE']
    # first is moved from 'A' to 'B'
    df_new.loc[df_new['dose']=='B', 'INCIDENCE'] = df_new.loc[df_new['dose']=='A', 'INCIDENCE']
    df_new.loc[df_new['dose']=='B', 'CUMULATIVE'] = df_new.loc[df_new['dose']=='A', 'CUMULATIVE']
    # 'A' becomes the empty category and is nullified
    df_new.loc[df_new['dose']=='A', 'INCIDENCE'] = 0
    df_new.loc[df_new['dose']=='A', 'CUMULATIVE'] = 0
    
    
    # Initialise rescaling parameter columns
    all_rescaling = ['E_susc_WT', 'E_inf_WT', 'E_hosp_WT', \
                      'E_susc_abc', 'E_inf_abc', 'E_hosp_abc', \
                      'E_susc_delta', 'E_inf_delta', 'E_hosp_delta', \
                      'E_susc',  'E_inf', 'E_hosp'] # averaged values
    for rescaling in all_rescaling:
        df_new[rescaling] = 0
        df_new.loc[df_new['dose']=='A', rescaling] = 1
    

    # Return to multiindex
    df_new = df_new.reset_index()
    if spatial:
        df_new = df_new.drop(columns=['age_class', 'population'])
        df_new = df_new.set_index(['date', 'NIS', 'age', 'dose'])
    else:
        df_new = df_new.drop(columns=['population'])
        df_new = df_new.set_index(['date',        'age', 'dose'])

    # rename indices to clearly understandable categories
    rename_indices = dict({'A' : 'none', 'B' : 'first', 'C' : 'full', 'E' : 'booster'})
    df_new = df_new.rename(index=rename_indices)
    
    # reset indices for next line of calculations
    df_new = df_new.reset_index()
    all_available_dates = df_new.date.unique()
    df_new = df_new.set_index(['date', 'dose']).sort_index()
    for rescaling in ['E_susc', 'E_inf', 'E_hosp']:
        for VOC in ['WT', 'abc', 'delta']:
            for dose in ['first', 'full', 'booster']:
                # Calculate E values for this rescaling type and dose
                onset_days_temp = onset_days[VOC][rescaling][dose]
                E_init_temp = E_init[VOC][rescaling][dose]
                E_best_temp = E_best[VOC][rescaling][dose]
                E_waned_temp = E_waned[VOC][rescaling][dose]
                for date in all_available_dates:
                    # run over all dates before this date
                    for d in all_available_dates[all_available_dates<=date]:
                        # Calculate how many days there are in between
                        delta_days = pd.Timedelta(date - d).days
                        # Sum over previous days with a weight depending on incidence, dose type, and waning of vaccines
                        weight = waning_exp_delay(delta_days, onset_days_temp, E_init_temp, E_best_temp, E_waned_temp)
                        df_new.loc[(date, dose), f'{rescaling}_{VOC}'] += df_new.loc[(d, dose),'INCIDENCE'].to_numpy() * weight
                    # normalise over total number of vaccinated subjects up to that point
                    df_new.loc[(date,dose), f'{rescaling}_{VOC}'] /= df_new.loc[(date,dose), 'CUMULATIVE']
            # Get rid of all division-by-zero results
            df_new.loc[df_new[f'{rescaling}_{VOC}']==np.inf, rescaling] = 1
            df_new[f'{rescaling}_{VOC}'].fillna(1, inplace=True)
        
    if spatial:
        df_new = df_new.groupby(['date', 'NIS', 'age', 'dose']).first()
    else:
        df_new = df_new.groupby(['date', 'age', 'dose']).first()
        
    # Calculate weighted average from fractions and current rescaling factor
    if spatial:
        all_available_NIS = initN.NIS.unique()
        for date in all_available_dates:
            for NIS in all_available_NIS:
                print("working on NIS ", NIS)
                for interval in intervals:
                    for rescaling in ['E_susc', 'E_inf', 'E_hosp']:
                        for VOC in ['WT', 'abc', 'delta']:
                            df_new.loc[(date,NIS,interval, 'weighted_sum'), f'{rescaling}_{VOC}'] = \
                                (df_new.loc[(date,NIS,interval), f'{rescaling}_{VOC}'] * \
                                 df_new.loc[(date,NIS,interval), 'fraction']).sum()
                        df_new.loc[(date,NIS,interval, 'weighted_sum'), rescaling] = 0
                        for i, VOC in enumerate(['WT', 'abc', 'delta']):
                            df_new.loc[(date,NIS,interval, 'weighted_sum'), rescaling] += \
                                (df_new.loc[(date,NIS,interval, 'weighted_sum'), f'{rescaling}_{VOC}']) * \
                                VOC_func(pd.Timestamp(date), 0, 0)[0][i]

    else:
        # Note: takes much longer because there is at least 7 times more data available
        for date in all_available_dates:
            for interval in intervals:
                for rescaling in ['E_susc', 'E_inf', 'E_hosp']:
                    for VOC in ['WT', 'abc', 'delta']:
                        df_new.loc[(date,interval, 'weighted_sum'), f'{rescaling}_{VOC}'] = \
                            (df_new.loc[(date,interval), f'{rescaling}_{VOC}'] * \
                             df_new.loc[(date,interval), 'fraction']).sum()
                    df_new.loc[(date,interval, 'weighted_sum'), rescaling] = 0
                    for i, VOC in enumerate(['WT', 'abc', 'delta']):
                        df_new.loc[(date,interval, 'weighted_sum'), rescaling] += \
                            (df_new.loc[(date,interval, 'weighted_sum'), f'{rescaling}_{VOC}']) * \
                            VOC_func(pd.Timestamp(date), 0, 0)[0][i]  
                    
    if spatial:
        df_new = df_new.groupby(['date', 'NIS', 'age', 'dose']).first()
    else:
        df_new = df_new.groupby(['date', 'age', 'dose']).first()
    
    return df_new

test_df = make_rescaling_dataframe(df_inc, initN, VOC_function)

In [ ]:
test_df

# test_df2 = test_df.drop(columns=['INCIDENCE', 'CUMULATIVE', 'E_susc_WT', 'E_inf_WT', 'E_hosp_WT', \
#                       'E_susc_abc', 'E_inf_abc', 'E_hosp_abc', \
#                       'E_susc_delta', 'E_inf_delta', 'E_hosp_delta'])

# test_df2 = test_df2.fillna(1)
# test_df2.to_csv('rescaling_values_vaccination.csv')

In [ ]:
# test_df.loc['2021-03-08', 10000, pd.Interval(45,55,closed='left'), 'weighted_sum']

date = '2021-03-08'
NIS = 10000
interval = pd.Interval(45,55,closed='left')
rescaling = 'E_susc'

# test_df.loc[(date,NIS,interval, 'weighted_sum'), rescaling] = \
#     test_df.loc[(date, NIS, interval, 'weighted_sum'), f'{rescaling}_WT'] * VOC_function(pd.Timestamp(date), 0, 0)[0][0]

# test_df.loc[(date,NIS,interval, 'weighted_sum'), rescaling]

# test_df.loc[(date, NIS, interval, 'weighted_sum'), f'{rescaling}_WT']

test_df.loc[(date,NIS,interval, 'weighted_sum'), rescaling] = (test_df.loc[(date, NIS, interval, 'weighted_sum'), f'{rescaling}_WT']).values

test_df.loc[(date,NIS,interval, 'weighted_sum'), rescaling]

# += \
#                             df_new.loc[(date,NIS,interval, 'weighted_sum'), f'{rescaling}_{VOC}'] * \
#                             VOC_func(pd.Timestamp(date), 0, 0)[0][i]